In [2]:
import pandas as pd

In [3]:
df_Rosja = pd.read_csv("/content/wysocki_jan.csv")

In [4]:
df_Rosja_temp = df_Rosja.head() #dobra praktyka

In [ ]:
print(df_Rosja)

In [ ]:
df_Rosja.head()

In [ ]:
df_Rosja.tail()

In [ ]:
df_Rosja.info()

In [ ]:
df_Rosja.describe()

In [ ]:
type(df_Rosja['created_at'])

1. Przetwarzanie i oczyszczanie danych


In [7]:
#1 Zamień skróty nazw dni tygodnia na polskie odpowiedniki
df_Rosja["data_utworzenia"] = [elem.replace("Mon", "Poniedziałek").replace("Tue", "Wtorek").replace("Wed", "Środa").replace("Thu", "Czwartek").replace("Fri", "Piątek").replace("Sat", "Sobota").replace("Sun", "Niedziela") for elem in df_Rosja["created_at"]]

In [ ]:
print(df_Rosja['data_utworzenia'])

In [ ]:
df_Rosja.head()

In [ ]:
#2 Zamień skróty nazw miesięcy na liczbowe odpowiedniki (np Jun na 06) 
df_Rosja['data_utworzenia_2'] = [elem.replace("Nov", "11") for elem in df_Rosja["data_utworzenia"]]
print(df_Rosja['data_utworzenia_2'])

#Mam tylko daty listopadowe dlatego zamieniłem tylko jeden miesiąc

In [ ]:
#3 Pobierz wszystkie linki do tweetów i przekaż je do listy.

linki = []

for elem in df_Rosja['tweet_url']:
    linki.append(elem)

print(linki)



In [ ]:
#4 Pobierz wszystkie linki znajdujące się w tweetach i przekaż je do listy (kolumna urls)

linki_w_tweetach = []

for elem in df_Rosja['urls']:
    linki_w_tweetach.append(elem)

print(linki_w_tweetach)


In [ ]:
#5 Pobierz wszystkie linki do obrazków i przekaż je do listy (kolumna media).

obrazki = []

for elem in df_Rosja['media']:
    obrazki.append(elem)

print(obrazki)

In [14]:
#6 Usuń wszystkie słowa oznaczone jako stopwords i przekaż oczyszczony tekst do nowej kolumny o nazwie text_without_stopwords
import spacy
nlp = spacy.load("pl_core_news_lg")
doc2 = str(df_Rosja['text'])

doc_rosja = nlp(doc2)

for token in doc_rosja:
  if not token.is_stop:
    tokens.append(token)

    





KeyboardInterrupt: ignored

2. Eksploracyjna analiza danych

In [ ]:
#1 Wypisz top5 tweetów z największa liczbą polubień.

df_Rosja[['text','favorite_count','id']].sort_values('favorite_count', ascending=False,).head(5)
 



In [ ]:
#2 Wypisz top5 tweetów z największa liczbą retweetów.

df_Rosja[['text','retweet_count','id']].sort_values('retweet_count', ascending=False,).head(5)
 

In [ ]:
#3 Wyświetl tylko te tweety, które nie są uznane jako “wrażliwe” (kolumna possibly_sensitive).

df_Rosja[['text','possibly_sensitive']].loc[df_Rosja['possibly_sensitive'] == False]

In [ ]:
#4 Wyświetl tweety użytkownika, który założył konto najwcześniej (spośród wszystkich użytkowników w datasecie) (kolumna user_created_at)

df_Rosja['user_created_at'] = pd.to_datetime(df_Rosja.user_created_at)

df_Rosja[['user_created_at','text','user_name']].loc[df_Rosja['user_created_at'] == df_Rosja['user_created_at'].min()]

In [ ]:
#5 Wyświetl tweety użytkownika. który ma najwięcej followersów.

df_Rosja[['user_followers_count','text','user_name']].loc[df_Rosja['user_followers_count'] == df_Rosja['user_followers_count'].max()]

In [ ]:
#6 Wyświetl tylko zweryfikowanych użytkowników (kolumna user_verified)

df_Rosja['user_name'].loc[df_Rosja['user_verified'] == True].drop_duplicates()

In [ ]:
#7 Podaj, w który dzień tygodnia najczęściej były publikowane tweety z datasetu

df_Rosja['dzien'] = df_Rosja['user_created_at'].dt.dayofweek

df_Rosja.groupby('dzien')['dzien'].count()

#Najwięcej twittów napsiano w niedzielę


Część 3 - przetwarzanie języka naturalnego


In [ ]:
!pip install spacy -U
!python -m spacy download pl_core_news_lg

import spacy

In [24]:
nlp = spacy.load("pl_core_news_lg")

In [25]:
lista_text = list(df_Rosja['text'])

In [26]:
string_text = ' '.join(str(e) for e in lista_text)

In [28]:
nlp.max_length = 1988086

In [29]:
doc_text = nlp(string_text)

In [30]:
persons = []
places = []
organizations = []



In [31]:
for ent in doc_text.ents:
  if ent.label == "orgName":
    organizations.append(ent)

In [32]:
for ent in doc_text.ents:
  if ent.label == "persName":
    persons.append(ent)

In [33]:
for ent in doc_text.ents:
  if ent.label == "placeName":
    places.append(ent)

In [ ]:
df_Rosja['persons'] = pd.Series(persons)

df_Rosja['organizations'] = pd.Series(organizations)

df_Rosja['places'] = pd.Series(places)

In [35]:
df_Rosja.to_csv('df_rosja.csv')